### Spark Operations explained in this notebook

- Reading Parquet File into a Spark DataFrame (With/Without Schema)
- Writing Spark DataFrame into a Parquet File
- Repartitioing a Spark DataFrame
- Adding Columns to a DataFrame
- Renaming Columns to a DataFrame
- Changing Datatype of a Spark DataFrame
- Joining two DataFrames
- Using GroupBy and Count in DataFrames

### SparkSQL Operations and Spark logic behind the scenes explained in this notebooks

- Registering DataFrame as a Table
- Using SQL statements to create new DataFrames
- Understanding how Group By works internally in Spark
  View DAG Visualizations to understand reshuflling 
  
- Understanding how Join works internally in Spark
  View DAG Visualizations to understand broadcast Exchange

### Importing Required Depeendencies

In [13]:
from datetime import datetime
from collections import namedtuple

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

### Starting Spark Session

In [3]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

22/06/09 08:33:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/09 08:33:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/09 08:33:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### Downloading Higher Volume Data Parquet File

In [4]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.parquet

### Creating Spark Dataframe

In [4]:
df = spark.read.parquet('fhvhv_tripdata_2021-01.parquet')
df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [5]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(originating_base_num,StringType,true),StructField(request_datetime,TimestampType,true),StructField(on_scene_datetime,TimestampType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropoff_datetime,TimestampType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(trip_miles,DoubleType,true),StructField(trip_time,LongType,true),StructField(base_passenger_fare,DoubleType,true),StructField(tolls,DoubleType,true),StructField(bcf,DoubleType,true),StructField(sales_tax,DoubleType,true),StructField(congestion_surcharge,DoubleType,true),StructField(airport_fee,DoubleType,true),StructField(tips,DoubleType,true),StructField(driver_pay,DoubleType,true),StructField(shared_request_flag,StringType,true),StructField(shared_match_flag,StringType,true),StructField(access_a_ride_flag,StringType,true),StructField(wav_req

 ### Repartitioning into smaller Files

In [7]:
df.repartition(24) \
    .write \
    .parquet('fhvhv_parquet/2021/01/')

In [8]:
df1 = spark.read.parquet('fhvhv_parquet/2021/01/')
df1.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [9]:
df1.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [14]:
df1 = df1 \
    .withColumn('PULocationID',col('PULocationID').cast('integer')) \
    .withColumn('DOLocationID',col('PULocationID').cast('integer'))

In [17]:
df1.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: strin

### Reading Higher Volume Data with pre-defined Schema

In [19]:
from pyspark.sql import types
schema = types.StructType([
  types.StructField('hvfhs_license_num', types.StringType(), True),
  types.StructField('dispatching_base_num', types.StringType(), True),
  types.StructField('originating_base_num', types.StringType(), True),
  types.StructField('request_datetime', types.TimestampType(), True),
  types.StructField('on_scene_datetime', types.TimestampType(), True),
  types.StructField('pickup_datetime', types.TimestampType(), True),
  types.StructField('dropoff_datetime', types.TimestampType(), True),
  types.StructField('PULocationID', types.LongType(), True),
  types.StructField('DOLocationID', types.LongType(), True),
  types.StructField('trip_miles', types.DoubleType(), True),
  types.StructField('trip_time', types.LongType(), True),
  types.StructField('base_passenger_fare', types.DoubleType(), True),
  types.StructField('tolls', types.DoubleType(), True),
  types.StructField('bcf', types.DoubleType(), True),
  types.StructField('sales_tax', types.DoubleType(), True),
  types.StructField('congestion_surcharge', types.DoubleType(), True),
  types.StructField('airport_fee', types.DoubleType(), True),
  types.StructField('tips', types.DoubleType(), True),
  types.StructField('driver_pay', types.DoubleType(), True),
  types.StructField('shared_request_flag', types.StringType(), True),
  types.StructField('shared_match_flag', types.StringType(), True),
  types.StructField('access_a_ride_flag', types.StringType(), True),
  types.StructField('wav_request_flag', types.StringType(), True),
  types.StructField('wav_match_flag', types.StringType(), True)
])

In [24]:
df_with_schema = spark.read.schema(schema).parquet('fhvhv_tripdata_2021-01.parquet')

In [25]:
df_with_schema.show(5)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [26]:
df2.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

### Reading and Repartitioning Yellow and Green Taxi Data

In [27]:
# Green and Yellow data has been downloaded already using download_data.sh bash file

In [27]:
df_green = spark.read \
                .option("headers", "true") \
                .parquet("data/raw/green/2020/01/")
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2019-12-18 15:52:30|  2019-12-18 15:54:39|                 N|       1.0|         264|         264|            5.0|          0.0|        3.5|  0.5|    0.

In [28]:
df_green.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [29]:
df_yellow = spark.read \
                .option("headers", "true") \
                .parquet("data/raw/yellow/2020/01/")
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2020-01-01 00:28:15|  2020-01-01 00:33:03|            1.0|          1.2|       1.0|                 N|         238|         239|           1|        6.0|  3.0|    0.5|      1.4

In [30]:
df_yellow.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)



In [42]:
years = [2020, 2021]
colors = ['green', 'yellow']

for color in colors:
    for year in years:
        for month in range(1,13):            
            input_path=f'data/raw/{color}/{year}/{month:02d}/'
            output_path=f'data/pq/{color}/{year}/{month:02d}/'
            
            print(f'read {input_path}')                        
            input_file = spark.read.parquet(input_path)
            
            print(f'Partitioning into {output_path}')
            input_file \
                    .repartition(4) \
                    .write.parquet(output_path)

read data/raw/green/2020/01/
Partitioning into data/pq/green/2020/01/


read data/raw/green/2020/02/
Partitioning into data/pq/green/2020/02/


read data/raw/green/2020/03/
Partitioning into data/pq/green/2020/03/
read data/raw/green/2020/04/
Partitioning into data/pq/green/2020/04/
read data/raw/green/2020/05/
Partitioning into data/pq/green/2020/05/
read data/raw/green/2020/06/
Partitioning into data/pq/green/2020/06/
read data/raw/green/2020/07/
Partitioning into data/pq/green/2020/07/
read data/raw/green/2020/08/
Partitioning into data/pq/green/2020/08/
read data/raw/green/2020/09/
Partitioning into data/pq/green/2020/09/
read data/raw/green/2020/10/
Partitioning into data/pq/green/2020/10/
read data/raw/green/2020/11/
Partitioning into data/pq/green/2020/11/
read data/raw/green/2020/12/
Partitioning into data/pq/green/2020/12/
read data/raw/green/2021/01/
Partitioning into data/pq/green/2021/01/
read data/raw/green/2021/02/
Partitioning into data/pq/green/2021/02/
read data/raw/green/2021/03/
Partitioning into data/pq/green/2021/03/
read data/raw/green/2021/04/
Partitioning into data/pq/green/2021/04/
read data/raw/green/

read data/raw/yellow/2020/02/
Partitioning into data/pq/yellow/2020/02/


read data/raw/yellow/2020/03/
Partitioning into data/pq/yellow/2020/03/


...

read data/raw/yellow/2021/08/
Partitioning into data/pq/yellow/2021/08/


read data/raw/yellow/2021/09/
Partitioning into data/pq/yellow/2021/09/


read data/raw/yellow/2021/10/
Partitioning into data/pq/yellow/2021/10/


read data/raw/yellow/2021/11/
Partitioning into data/pq/yellow/2021/11/


read data/raw/yellow/2021/12/
Partitioning into data/pq/yellow/2021/12/


### Combining Yellow and Green Taxi Data

In [50]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [52]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [58]:
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

In [60]:
df_green.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [59]:
df_yellow.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee']

In [61]:
common_columns = []

for col in df_green.columns:
    if col in set(df_yellow.columns):
        common_columns.append(col)
        
common_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [62]:
from pyspark.sql import functions as F

In [64]:
df_green_sel = df_green \
    .select(common_columns) \
    .withColumn('service_type', F.lit('green'))

In [65]:
df_yellow_sel = df_yellow \
    .select(common_columns) \
    .withColumn('service_type', F.lit('yellow'))

In [66]:
df_yellow_sel.show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|service_type|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|       1|2020-01-02 18:49:18|2020-01-02 19:06:24|                 N|       1.0|         263|         230|            0.0|          2.7|       13.0|  3.5|    0.5|      3.45|         0.0|       

In [67]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [68]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



### Using Spark SQL

In [50]:
# Register Dataframe as a Table
df_trips_data.registerTempTable('trips_data')

NameError: name 'df_trips_data' is not defined

In [73]:
spark.sql("""
SELECT 
    service_type,
    COUNT(1) AS count
FROM 
    trips_data 
GROUP BY 
    1;
""").show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [79]:
df_result = spark.sql(
"""
SELECT
  -- Revenue Grouping
  PULocationID AS revenue_zone,
  date_trunc('month', pickup_datetime) AS revenue_month,
  service_type,

  -- Revenue Calculation
  SUM(fare_amount) AS revenue_monthly_fare,
  SUM(extra) AS revenue_monthly_extra,
  SUM(mta_tax) AS revenue_monthly_mta_tax,
  SUM(tip_amount) AS revenue_monthly_tip_amount,
  SUM(tolls_amount) AS revenue_monthly_tolls_amount,
  SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
  SUM(total_amount) AS revenue_monthly_total_amount,
  SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

  -- Aggregated Counts
  AVG(passenger_count) AS avg_monthly_passenger_count,
  AVG(trip_distance) AS avg_monthly_trip_distance

  FROM tripS_data
  GROUP BY 1, 2, 3
""")

In [80]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+---------------------------+-------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_monthly_passenger_count|avg_monthly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+---------------------------+-------------------------+
|         232

In [82]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')

### Understanding GROUP BY in Spark

In [9]:
df_green.registerTempTable('green')

In [49]:
df_green_revenue = spark.sql(
"""
SELECT
  -- Revenue Grouping
  date_trunc('hour', lpep_pickup_datetime) AS hour,
  PULocationID AS zone,  

  -- Revenue Calculation
  SUM(total_amount) AS amount,
  COUNT(1) AS number_records

FROM 
    green
WHERE 
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 
    1, 2
""")

In [50]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [51]:
df_yellow.registerTempTable('yellow')

In [52]:
df_yellow_revenue = spark.sql(
"""
SELECT
  -- Revenue Grouping
  date_trunc('hour', tpep_pickup_datetime) AS hour,
  PULocationID AS zone,  

  -- Revenue Calculation
  SUM(total_amount) AS amount,
  COUNT(1) AS number_records

FROM 
    yellow
WHERE 
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 
    1, 2
""")

In [53]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [20]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-14 14:00:00| 138|14898.500000000015|           339|
|2020-01-11 22:00:00| 132|18350.820000000025|           355|
|2020-01-22 16:00:00| 230| 9063.570000000018|           507|
|2020-01-12 02:00:00|   4|1224.3200000000002|            71|
|2020-01-13 11:00:00| 237| 7470.460000000019|           550|
|2020-01-04 17:00:00| 143|2078.5999999999995|           135|
|2020-01-28 19:00:00| 238| 3897.219999999997|           247|
|2020-01-08 08:00:00|  68| 3530.239999999997|           223|
|2020-01-20 13:00:00| 163| 6030.090000000006|           345|
|2020-01-21 08:00:00| 113| 3592.989999999998|           225|
|2020-01-20 19:00:00|  48|4079.8999999999974|           262|
|2020-01-26 12:00:00| 113| 2891.249999999999|           196|
|2020-01-21 20:00:00| 132|26911.159999999985|           498|
|2020-01-14 13:00:00| 23

### Understanding JOIN in Spark

In [45]:
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')
df_green_revenue = spark.read.parquet('data/report/revenue/green')

In [54]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')
    
df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [55]:
df_join = df_yellow_revenue_tmp.join(df_green_revenue_tmp, on=['hour', 'zone'], how='outer')

In [22]:
df_join.show()

+-------------------+----+------------------+---------------------+------------------+--------------------+
|               hour|zone|     yellow_amount|yellow_number_records|      green_amount|green_number_records|
+-------------------+----+------------------+---------------------+------------------+--------------------+
|2020-01-01 01:00:00|  24| 783.9000000000002|                   44|            126.42|                   6|
|2020-01-01 01:00:00| 242|              null|                 null|             30.94|                   2|
|2020-01-01 04:00:00| 202|              42.1|                    2|              null|                null|
|2020-01-01 05:00:00|  71|             49.71|                    1|              null|                null|
|2020-01-01 05:00:00| 244|185.16999999999996|                   17|             25.26|                   2|
|2020-01-01 07:00:00|  48|2857.2099999999973|                  120|              null|                null|
|2020-01-01 07:00:00| 112|  

In [56]:
df_join \
    .write.parquet('data/report/revenue/total', mode='overwrite')

In [28]:
df_join = spark.read.parquet('data/report/revenue/total')

In [29]:
df_join.show()

+-------------------+----+------------------+---------------------+------------------+--------------------+
|               hour|zone|     yellow_amount|yellow_number_records|      green_amount|green_number_records|
+-------------------+----+------------------+---------------------+------------------+--------------------+
|2020-01-01 00:00:00| 233|2028.8199999999983|                  121|              null|                null|
|2020-01-01 01:00:00| 159|148.89999999999998|                    8|33.900000000000006|                   3|
|2020-01-01 03:00:00| 162|2134.1399999999985|                  126|              null|                null|
|2020-01-01 03:00:00| 196|              34.1|                    2|             34.66|                   2|
|2020-01-01 05:00:00|  56|             73.67|                    1|              8.84|                   1|
|2020-01-01 05:00:00|  68|1825.1899999999994|                   90|              null|                null|
|2020-01-01 05:00:00| 237|40

### Joining Zone File

In [37]:
df_zones = spark.read.parquet('zones')
df_zones = df_zones \
    .withColumnRenamed('Zone', 'Zone_name')
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|           Zone_name|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [38]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [39]:
df_result.show()

+-------------------+----+------------------+---------------------+------------------+--------------------+----------+---------+--------------------+------------+
|               hour|zone|     yellow_amount|yellow_number_records|      green_amount|green_number_records|LocationID|  Borough|           Zone_name|service_zone|
+-------------------+----+------------------+---------------------+------------------+--------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00| 233|2028.8199999999983|                  121|              null|                null|       233|Manhattan| UN/Turtle Bay South| Yellow Zone|
|2020-01-01 01:00:00| 159|148.89999999999998|                    8|33.900000000000006|                   3|       159|    Bronx|       Melrose South|   Boro Zone|
|2020-01-01 03:00:00| 162|2134.1399999999985|                  126|              null|                null|       162|Manhattan|        Midtown East| Yellow Zone|
|2020-01-01 03:00:00| 

In [44]:
df_result.drop('zone').write.parquet('tmp/revenue-zones', mode='overwrite')

In [43]:
df_result_tmp = spark.read.parquet('tmp/revenue-zones/')
df_result_tmp.show()

+-------------------+------------------+---------------------+------------------+--------------------+----------+---------+--------------------+------------+
|               hour|     yellow_amount|yellow_number_records|      green_amount|green_number_records|LocationID|  Borough|           Zone_name|service_zone|
+-------------------+------------------+---------------------+------------------+--------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|2028.8199999999983|                  121|              null|                null|       233|Manhattan| UN/Turtle Bay South| Yellow Zone|
|2020-01-01 01:00:00|148.89999999999998|                    8|33.900000000000006|                   3|       159|    Bronx|       Melrose South|   Boro Zone|
|2020-01-01 03:00:00|2134.1399999999985|                  126|              null|                null|       162|Manhattan|        Midtown East| Yellow Zone|
|2020-01-01 03:00:00|              34.1|            